>Get the model :

In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/GATConv.1.model.single_batch.2607.pt \
/media/concha-eloko/Linux/PPT_clean/ficheros_28032023

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/GATv2.1.model.single_batch.2707.pt \
/media/concha-eloko/Linux/PPT_clean/ficheros_28032023


In [1]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv, HeteroConv , GATConv, GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader

import torch
from torch import nn 
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
warnings.filterwarnings("ignore")


In [2]:
path_work = "/media/concha-eloko/Linux/PPT_clean"

    # Open the DF
DF_info = pd.read_csv(f"{path_work}/DF_Dpo.final.2705.tsv", sep = "\t" ,  header = 0 )
    # Open the embeddings
DF_embeddings = pd.read_csv(f"{path_work}/Dpo.2705.embeddings.ultimate.csv", sep = ",", header= None )
DF_embeddings.rename(columns={0: 'index'}, inplace=True)

    # Filter the DF :
DF_info_filtered = DF_info[~DF_info["KL_type_LCA"].str.contains("\\|")]
DF_info_ToReLabel = DF_info[DF_info["KL_type_LCA"].str.contains("\\|")]
all_data = pd.merge(DF_info_filtered , DF_embeddings , on = "index")

# Mind the over representation of outbreaks :
all_data = all_data.drop_duplicates(subset = ["Infected_ancestor","index","prophage_id"] , keep = "first").reset_index(drop=True)
all_data



,Phage,KL_type_LCA,Infected_ancestor,Protein_name,Dataset,index,seq,prophage_id,1,2,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,GCF_016651625.1__phage29,KL43,GCF_016651625.1,GCF_016651625.1__phage29__142,ppt,ppt__1,MSVPNQTPYNIYTANGLTTVFTYEFYIISASDLRVSINGDVVTSGY...,prophage_12186,-0.018416,0.022387,...,0.030484,0.004437,0.087907,0.015800,0.025778,0.065790,0.034045,-0.070899,0.016068,0.065339
1,GCF_016651625.1__phage29,KL43,GCF_016651625.1,GCF_016651625.1__phage29__150,anubis,anubis__0,MRANLIKTNFTAGEISPRLMGRVDIARYANGAKIIENAVCVVQGGV...,prophage_12186,0.036016,0.005938,...,0.008465,-0.037612,0.008772,0.010556,-0.049738,-0.012549,0.092624,-0.136602,-0.191378,0.135658
2,GCF_016651625.1__phage12,KL43,GCF_016651625.1,GCF_016651625.1__phage12__59,ppt,ppt__4,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...,prophage_924,0.026004,0.024372,...,0.054964,-0.026018,0.018206,0.036751,-0.032549,0.064112,0.061520,-0.024423,-0.027998,0.028089
3,GCF_019928025.1__phage0,KL43,n1471,GCF_019928025.1__phage0__10,ppt,ppt__4,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...,prophage_2929,0.026004,0.024372,...,0.054964,-0.026018,0.018206,0.036751,-0.032549,0.064112,0.061520,-0.024423,-0.027998,0.028089
4,GCF_004313505.1__phage4,KL14,GCF_004313505.1,GCF_004313505.1__phage4__113,anubis,anubis__5,MSEYDTGNPVPSASMPDAWDNMQSIDKFVNSSDETITTRTGQQLDT...,prophage_11091,-0.023648,0.052674,...,-0.003019,-0.025991,0.068538,-0.051192,0.026481,0.069100,0.017813,-0.103797,0.018961,0.117058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9672,GCF_015209405.1__phage17,KL2,81.9/1004099,GCF_015209405.1__phage17__0,anubis,anubis__1643,MASIKELPRWEDEVYQIARGDKVEGGVGGIANMQAKTLAERTRYLK...,prophage_12,-0.014377,0.119705,...,0.040801,-0.001167,0.108840,0.008335,0.026747,0.046616,0.033648,-0.092224,0.048720,0.034757
9673,GCF_004311345.1__phage11,KL34,GCF_004311345.1,GCF_004311345.1__phage11__99,anubis,anubis__1644,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...,prophage_11944,-0.001691,-0.067717,...,0.119783,0.073706,0.057788,0.004341,0.007389,-0.081588,0.100995,-0.045545,0.021685,0.013167
9674,GCF_900172635.1__phage3,KL124,GCF_900172635.1,GCF_900172635.1__phage3__1811,anubis,anubis__1647,MTANYPASILPPNATAVERAIDRASAAALERLPVYLIRWVKDPDSC...,prophage_11522,-0.002816,-0.055336,...,0.110956,0.056314,0.059700,0.004641,0.016676,-0.071711,0.095491,-0.055784,0.015615,0.000201
9675,GCF_013604525.1__phage18,KL30,n49324932,GCF_013604525.1__phage18__52,anubis,anubis__1649,MANIEKLGSSSPEVLLKNATNLDKLVNGRESESLPDRFGVLRKTWH...,prophage_2576,-0.000031,0.046085,...,-0.016720,-0.006530,0.077754,-0.024675,0.026408,0.016113,0.043904,-0.091418,0.078778,0.105842


In [3]:
just_chekin = all_data.drop_duplicates(subset = ["Infected_ancestor"] , keep = "first").reset_index(drop=True)


In [13]:
instances_bacteria = all_data.drop_duplicates(subset = ["KL_type_LCA"] , keep = "first").reset_index(drop=True)
instances_bacteria

,Phage,KL_type_LCA,Infected_ancestor,Protein_name,Dataset,index,seq,prophage_id,1,2,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,GCF_016651625.1__phage29,KL43,GCF_016651625.1,GCF_016651625.1__phage29__142,ppt,ppt__1,MSVPNQTPYNIYTANGLTTVFTYEFYIISASDLRVSINGDVVTSGY...,prophage_12186,-0.018416,0.022387,...,0.030484,0.004437,0.087907,0.015800,0.025778,0.065790,0.034045,-0.070899,0.016068,0.065339
1,GCF_004313505.1__phage4,KL14,GCF_004313505.1,GCF_004313505.1__phage4__113,anubis,anubis__5,MSEYDTGNPVPSASMPDAWDNMQSIDKFVNSSDETITTRTGQQLDT...,prophage_11091,-0.023648,0.052674,...,-0.003019,-0.025991,0.068538,-0.051192,0.026481,0.069100,0.017813,-0.103797,0.018961,0.117058
2,GCF_902156295.1__phage7,KL107,n29132913,GCF_902156295.1__phage7__1791,ppt,ppt__21,MAFVTTQDGVNIYFKDWGPKEAQPIVFHHGWPLSADDWDNQMLFFL...,prophage_15,0.035789,0.004226,...,0.038306,-0.078897,-0.061039,0.000776,-0.054214,0.027240,0.044547,-0.133716,-0.016950,0.110996
3,GCF_001902965.4__phage5,KL106,n3199,GCF_001902965.4__phage5__17,ppt,ppt__21,MAFVTTQDGVNIYFKDWGPKEAQPIVFHHGWPLSADDWDNQMLFFL...,prophage_15,0.035789,0.004226,...,0.038306,-0.078897,-0.061039,0.000776,-0.054214,0.027240,0.044547,-0.133716,-0.016950,0.110996
4,GCF_019703125.1__phage5,KL23,n3720,GCF_019703125.1__phage5__1686,ppt,ppt__21,MAFVTTQDGVNIYFKDWGPKEAQPIVFHHGWPLSADDWDNQMLFFL...,prophage_15,0.035789,0.004226,...,0.038306,-0.078897,-0.061039,0.000776,-0.054214,0.027240,0.044547,-0.133716,-0.016950,0.110996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,GCF_001631645.1__phage25,KL42,GCF_001631645.1,GCF_001631645.1__phage25__67,anubis,anubis__1279,MPIQQLPLMKGLGKDFKNADYIDYLPVNMLATPKEVLNSNGYLRSF...,prophage_11946,0.027796,0.063570,...,0.011792,-0.043679,0.038427,0.046187,-0.016313,0.034837,0.105800,-0.068054,-0.118124,0.107679
123,GCF_021261925.1__phage1,KL143,GCF_021261925.1,GCF_021261925.1__phage1__95,anubis,anubis__1302,MSTIIEKKQWENGVSLLRRSDKVEGGVGGKANEQAQQLANRTQYLK...,prophage_7784,0.007979,0.045936,...,0.025863,0.022980,0.039756,-0.020012,-0.003264,0.034406,0.062960,-0.090890,-0.064428,0.106303
124,GCF_002187665.1__phage4,KL154,GCF_002187665.1,GCF_002187665.1__phage4__81,anubis,anubis__964,MANFTSSLISDSASESNQAGWWHPLDSFQGAEYYGICKEYGVAGKH...,prophage_12700,-0.042222,0.007047,...,0.076963,0.002725,-0.001169,-0.013834,0.002876,-0.038183,0.038725,-0.185250,-0.067399,0.109580
125,GCF_018275035.1__phage20,KL144,GCF_018275035.1,GCF_018275035.1__phage20__0,anubis,anubis__1118,MHQPSVRLSSRLLSYLISVLLAGQPLLPALGAAIAPQGATGVDSAA...,prophage_13247,-0.085853,-0.088854,...,0.005364,0.016892,-0.079329,-0.028781,-0.079868,-0.013301,0.123361,-0.038579,-0.081845,0.038455


In [18]:
instances_bacteria = all_data.drop_duplicates(subset = ["KL_type_LCA"] , keep = "first").reset_index(drop=True)
index_interest = []
for ancestor in instances_bacteria.Infected_ancestor :
    index = ID_nodes_A[ancestor]
    index_interest.append(index)

index_interest

[0,
 2,
 4,
 7,
 64,
 134,
 242,
 251,
 289,
 290,
 292,
 297,
 315,
 323,
 324,
 330,
 337,
 344,
 348,
 351,
 376,
 418,
 438,
 454,
 471,
 499,
 508,
 578,
 633,
 635,
 737,
 743,
 747,
 750,
 753,
 757,
 761,
 762,
 889,
 899,
 936,
 1007,
 1025,
 1030,
 1053,
 1056,
 1060,
 1061,
 1066,
 1094,
 1103,
 1117,
 1133,
 1167,
 1378,
 1384,
 1413,
 1417,
 1554,
 1567,
 1586,
 1670,
 1673,
 1675,
 1677,
 1689,
 1692,
 1712,
 1775,
 1782,
 1785,
 1830,
 1863,
 1917,
 1938,
 1939,
 1949,
 1993,
 1996,
 2010,
 2044,
 2079,
 2087,
 2107,
 2138,
 2149,
 2158,
 2164,
 2185,
 2238,
 2253,
 2254,
 2289,
 2292,
 2320,
 2344,
 2391,
 2402,
 2407,
 2454,
 2469,
 2480,
 2493,
 2578,
 2593,
 2604,
 2639,
 2667,
 2684,
 2727,
 2807,
 2880,
 2944,
 2946,
 3003,
 3070,
 3227,
 3345,
 3355,
 3406,
 3467,
 3472,
 3545,
 3557,
 4402,
 4429,
 4444]

In [22]:
graph_data.x_dict["A"]

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [21]:
tenosor_interest = [graph_data.x_dict["A"][i] for _,i in enumerate(index_interest)]

[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [11]:
indexation_nodes_A = all_data["Infected_ancestor"].unique().tolist()  
indexation_nodes_B1 = all_data["Phage"].unique().tolist() + [f"Dpo_to_predict_{n}" for n in DF_embeddings["index"].unique().tolist()]
indexation_nodes_B2 = DF_embeddings["index"].unique().tolist() 

ID_nodes_A = {item:index for index, item in enumerate(indexation_nodes_A)}
ID_nodes_A_r = {index:item for index, item in enumerate(indexation_nodes_A)}

ID_nodes_B1 = {item:index for index, item in enumerate(indexation_nodes_B1)}
ID_nodes_B1_r = {index:item for index, item in enumerate(indexation_nodes_B1)}

ID_nodes_B2 = {item:index for index, item in enumerate(indexation_nodes_B2)}
ID_nodes_B2_r = {index:item for index, item in enumerate(indexation_nodes_B2)}



In [7]:
# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
graph_data = torch.load(f'{path_work}/graph_file.2607.OHE.pt')
graph_data

HeteroData(
  A={ x=[4530, 127] },
  B1={ x=[11339, 0] },
  B2={ x=[3608, 1280] },
  (B1, infects, A)={
    edge_index=[2, 7731],
    y=[7731]
  },
  (B2, expressed, B1)={
    edge_index=[2, 13285],
    y=[13285]
  },
  (A, harbors, B1)={
    edge_index=[2, 7731],
    y=[7731]
  }
)

In [5]:
class GNN(torch.nn.Module):
    def __init__(self, edge_type , conv, hidden_channels, heads, dropout): 
        super().__init__()
        self.conv = conv((-1,-1), hidden_channels, add_self_loops = False, heads = heads, dropout = dropout, shared_weights = True)
        self.hetero_conv = HeteroConv({edge_type: self.conv})
    def forward(self, x_dict, edge_index_dict):
        x = self.hetero_conv(x_dict, edge_index_dict)
        return x

# Classifier, Binary :
class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        self.lin1 = torch.nn.Linear(heads*hidden_channels + 127, 512)
        self.lin2 = torch.nn.Linear(512, 1)

    def forward(self, x_dict_A , x_dict_B1, graph_data):
        edge_type = ("B1", "infects", "A")
        edge_feat_A = x_dict_A["A"][graph_data[edge_type].edge_label_index[1]]
        edge_feat_B1 = x_dict_B1["B1"][graph_data[edge_type].edge_label_index[0]]
        features_phage = torch.cat((edge_feat_A ,edge_feat_B1), dim=-1)
        x = self.lin1(features_phage).relu()
        x = self.lin2(x)
        return x.view(-1)

class Model(torch.nn.Module):
    def __init__(self, conv, hidden_channels, heads, dropout):
        super().__init__()
        self.single_layer_model = GNN(("B2", "expressed", "B1") ,conv, hidden_channels,heads,dropout)
        self.EdgeDecoder = EdgeDecoder(hidden_channels,heads)

    def forward(self, graph_data):
        b1_nodes = self.single_layer_model(graph_data.x_dict , graph_data.edge_index_dict)
        a_nodes =  graph_data.x_dict
        out = self.EdgeDecoder(a_nodes ,b1_nodes , graph_data)
        return out


In [6]:
@torch.no_grad()
def make_predictions(model, data):
    model.eval() 
    output = model(data)
    probabilities = torch.sigmoid(output)  # Convert output to probabilities
    predictions = probabilities.round()  # Convert probabilities to class labels
    return predictions, probabilities

# Load the saved model
hidden_channels = 1000
conv = GATv2Conv
heads = 1
dropout = 0.1
model = Model( conv, hidden_channels, heads, dropout)
model.load_state_dict(torch.load(f"{path_work}/ficheros_28032023/GATv2.1.model.single_batch.2707.pt"))


<All keys matched successfully>

> Make the edges match 

In [ ]:
def get_nodes_id(B1A_index_file) :
    B1A_index_file = B1A_index_file.numpy()
    B1A_index_file = tuple(zip(B1A_index_file[0],B1A_index_file[1]))
    id_file = [(ID_nodes_B1_r[tup[0]] , ID_nodes_A_r[tup[1]]) for tup in B1A_index_file]
    return id_file

In [24]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
path_Dpo_f = f"{path_project}/rbp_work/77_full_Dpo"
path_Dpo_d = f"{path_project}/rbp_work/77_domain_Dpo"
path_pdb = f"{path_project}/77_phages_out"
path_Dpo_domain_org = "/media/concha-eloko/Linux/depolymerase_building/clean_77_phages_depo"

dpo_embeddings = pd.read_csv(f"{path_project}/rbp_work/Dpo_domains_77.esm2.embedding.csv", sep = "," , header = None)
dpo_embeddings = dpo_embeddings.drop([1281] , axis = 1)
dpo_embeddings.set_index([0], inplace = True)

In [28]:
dpo_embeddings["phage"] = dpo_embeddings.index.map(lambda x: x.split("__")[0])

In [54]:
# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
graph_data = torch.load(f'{path_work}/graph_file.2607.OHE.pt')


> Functions : 

In [ ]:
# Getting the nodes A features : 
instances_bacteria = all_data.drop_duplicates(subset = ["KL_type_LCA"] , keep = "first").reset_index(drop=True)
index_interest = []
for ancestor in instances_bacteria.Infected_ancestor :
    index = ID_nodes_A[ancestor]
    index_interest.append(index)
    
tensor_interest = [graph_data.x_dict["A"][i] for _,i in enumerate(index_interest)]
stacked_tensor = torch.stack(tensor_interest)


def graph_single_Dpo_pred(df_embeddings) : 
    pred_data_single = HeteroData()
    # Defining the nodes :
    l_dpos = len(df_embeddings)
    pred_data_single["A"].x = stacked_tensor
    pred_data_single["B1"].x = torch.empty((l_dpos, 0))
    pred_data_single["B2"].x = torch.tensor(df_embeddings.iloc[:, :1280].values , dtype=torch.float)
    # Defining the edge_file :
    edge_index_B2_B1 = torch.tensor([[i , i] for i in range(l_dpos)] , dtype=torch.long)
    pred_data_single['B2', 'expressed', 'B1'].edge_index = edge_index_B2_B1.t().contiguous()
    edge_index_B1_A = torch.tensor([[i,j] for i in range(l_dpos) for j in range(len(pred_data_single["A"].x))] , dtype=torch.long)
    pred_data_single['B1', 'infects', 'A'].edge_label_index = edge_index_B1_A.t().contiguous()
    return pred_data_single

def graph_multi_Dpo_pred(df_embeddings) : 
    pred_data_multi = HeteroData()
    # Defining the nodes :
    l_dpos = len(df_embeddings["phage"].unique())
    pred_data_multi["A"].x = stacked_tensor
    pred_data_multi["B1"].x = torch.empty((l_dpos, 0))
    pred_data_multi["B2"].x = torch.tensor(df_embeddings.iloc[:, :1280].values , dtype=torch.float)
    # Defining the edge  :
    dico_B1 = {phage : i for i,phage in enumerate(df_embeddings["phage"].unique())}
    edge_index_B2_B1 = torch.tensor([[i,dico_B1[phage]] for i,phage in enumerate(df_embeddings["phage"])] , dtype=torch.long)
    pred_data_multi['B2', 'expressed', 'B1'].edge_index = edge_index_B2_B1.t().contiguous()
    edge_index_B1_A = torch.tensor([[i,j] for i in range(l_dpos) for j in range(len(pred_data_multi["A"].x))] , dtype=torch.long)
    pred_data_multi['B1', 'infects', 'A'].edge_label_index = edge_index_B1_A.t().contiguous()
    return pred_data_multi


> Building the prediction graph : 

In [76]:
# Getting the nodes A features : 
instances_bacteria = all_data.drop_duplicates(subset = ["KL_type_LCA"] , keep = "first").reset_index(drop=True)
index_interest = []
for ancestor in instances_bacteria.Infected_ancestor :
    index = ID_nodes_A[ancestor]
    index_interest.append(index)
    
tensor_interest = [graph_data.x_dict["A"][i] for _,i in enumerate(index_interest)]
stacked_tensor = torch.stack(tensor_interest)

dico_kltype = {tuple(graph_data.x_dict["A"][i].numpy()) : all_data[all_data["Infected_ancestor"] == ID_nodes_A_r[i]]["KL_type_LCA"].values[0]  for _,i in enumerate(index_interest)}

In [79]:
tensor_interest[2]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0])

> Single Dpo mode : 

In [53]:
# Building the graph prediction : 
pred_data_single = HeteroData()

# Defining the nodes :
l_dpos = len(dpo_embeddings)
pred_data_single["A"].x = stacked_tensor
pred_data_single["B1"].x = torch.empty((l_dpos, 0))
pred_data_single["B2"].x = torch.tensor(dpo_embeddings.iloc[:, :1280].values , dtype=torch.float)

# Defining the edge_file :
edge_index_B2_B1 = torch.tensor([[i , i] for i in range(l_dpos)] , dtype=torch.long)
pred_data_single['B2', 'expressed', 'B1'].edge_index = edge_index_B2_B1.t().contiguous()
edge_index_B1_A = torch.tensor([[i,j] for i in range(l_dpos) for j in range(len(pred_data_single["A"].x))] , dtype=torch.long)
pred_data_single['B1', 'infects', 'A'].edge_label_index = edge_index_B1_A.t().contiguous()



> Multi Dpo mode : 

In [51]:
# Building the graph prediction : 
pred_data_multi = HeteroData()

# Defining the nodes :
l_dpos = len(dpo_embeddings["phage"].unique())
pred_data_multi["A"].x = stacked_tensor
pred_data_multi["B1"].x = torch.empty((l_dpos, 0))
pred_data_multi["B2"].x = torch.tensor(dpo_embeddings.iloc[:, :1280].values , dtype=torch.float)

# Defining the edge  :
dico_B1 = {phage : i for i,phage in enumerate(dpo_embeddings["phage"].unique())}
edge_index_B2_B1 = torch.tensor([[i,dico_B1[phage]] for i,phage in enumerate(dpo_embeddings["phage"])] , dtype=torch.long)
pred_data_multi['B2', 'expressed', 'B1'].edge_index = edge_index_B2_B1.t().contiguous()

edge_index_B1_A = torch.tensor([[i,j] for i in range(l_dpos) for j in range(len(pred_data_multi["A"].x))] , dtype=torch.long)
pred_data_multi['B1', 'infects', 'A'].edge_label_index = edge_index_B1_A.t().contiguous()


In [96]:
pred_data_single

HeteroData(
  A={ x=[127, 127] },
  B1={ x=[75, 0] },
  B2={ x=[75, 1280] },
  (B2, expressed, B1)={ edge_index=[2, 75] },
  (B1, infects, A)={ edge_label_index=[2, 9525] }
)

In [97]:
pred_data_multi

HeteroData(
  A={ x=[127, 127] },
  B1={ x=[53, 0] },
  B2={ x=[75, 1280] },
  (B2, expressed, B1)={ edge_index=[2, 75] },
  (B1, infects, A)={ edge_label_index=[2, 6731] }
)

***
### Make predictions : 

> Single Dpo

In [95]:
import json

class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.float32):
            return float(obj)
        return json.JSONEncoder.default(self, obj)
        
def get_nodes_id_single(B1A_index_file) :
    B1A_index_file = B1A_index_file.numpy()
    B1A_index_file = tuple(zip(B1A_index_file[0],B1A_index_file[1]))
    id_file = [(dpo_embeddings.index[tup[0]] , dico_kltype[tuple(tensor_interest[tup[1]].numpy())]) for tup in B1A_index_file]
    return id_file
    
with open(f"{path_work}/77_Dpos.PPT_pred.json", "w") as outfile :
    clean_results = {}
    predictions, probabilities = make_predictions(model, pred_data_single)
    ids = get_nodes_id_single(pred_data_single[("B1", "infects", "A")].edge_label_index)
    results = tuple(zip(ids,predictions.numpy(),probabilities.numpy()))
    positive_results = [pred for pred in results if int(pred[1]) == 1]
    for pos_res in positive_results : 
        prot = pos_res[0][0]
        kltype = pos_res[0][1]
        score = pos_res[2]
        a = {}
        a[kltype] = score
        if score > 0.50 : 
            #a = {"KLtype" : kltype , "Score" : score}
            if prot not in clean_results : 
                clean_results[prot] = a
            else :
                clean_results[prot].update(a)
    json.dump(clean_results , outfile, cls=CustomEncoder)

import pprint
pp = pprint.PrettyPrinter(width = 250, sort_dicts = True, compact = True)
pp.pprint(clean_results)


{'K10PH82C1__cds_50_Dpo_domain': {'KL104': 0.6618506,
                                  'KL109': 0.63262224,
                                  'KL110': 0.8293527,
                                  'KL114': 0.69601125,
                                  'KL119': 0.97432804,
                                  'KL12': 0.6707964,
                                  'KL124': 0.5078355,
                                  'KL127': 0.85204,
                                  'KL134': 0.78653544,
                                  'KL137': 0.5425276,
                                  'KL139': 0.8297305,
                                  'KL14': 0.60890526,
                                  'KL141': 0.9762136,
                                  'KL142': 0.73725414,
                                  'KL145': 0.9240038,
                                  'KL150': 0.6304,
                                  'KL152': 0.9422726,
                                  'KL153': 0.6257279,
                             

In [84]:
import json

class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.float32):
            return float(obj)
        return json.JSONEncoder.default(self, obj)
        
def get_nodes_id_multi(B1A_index_file) :
    B1A_index_file = B1A_index_file.numpy()
    B1A_index_file = tuple(zip(B1A_index_file[0],B1A_index_file[1]))
    id_file = [(dpo_embeddings.phage[tup[0]] , dico_kltype[tuple(tensor_interest[tup[1]].numpy())]) for tup in B1A_index_file]
    return id_file
    
with open(f"{path_work}/77_phages.PPT_pred.json", "w") as outfile :
    clean_results = {}
    predictions, probabilities = make_predictions(model, pred_data_single)
    ids = get_nodes_id_single(pred_data_single[("B1", "infects", "A")].edge_label_index)
    results = tuple(zip(ids,predictions.numpy(),probabilities.numpy()))
    positive_results = [pred for pred in results if int(pred[1]) == 1]
    for pos_res in positive_results : 
        prot = pos_res[0][0]
        kltype = pos_res[0][1]
        score = pos_res[2]
        a = {}
        a[kltype] = score
        if score > 0.50 : 
            #a = {"KLtype" : kltype , "Score" : score}
            if prot not in clean_results : 
                clean_results[prot] = a
            else :
                clean_results[prot].update(a)
    json.dump(clean_results , outfile, cls=CustomEncoder)

import pprint
pp = pprint.PrettyPrinter(width = 250, sort_dicts = True, compact = True)
pp.pprint(clean_results)

tensor([0., 0., 0.,  ..., 0., 0., 1.])

In [42]:
pred_data

HeteroData(
  A={ x=[4530, 127] },
  B1={ x=[75, 0] },
  B2={ x=[75, 1280] },
  (B2, expressed, B1)={ edge_index=[2, 75] },
  (B1, infects, A)={ edge_index=[2, 339750] }
)